In [112]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

categoryDF=pd.read_csv("Z:/MSBooks/NLP/Project/aspectCategoryDetection.csv",delimiter='#',encoding = 'utf-8')
categoryDF.head()
train, test = train_test_split(categoryDF, test_size = 0.2)
y_train = train.aspectCategory.unique()
labels = y_train.tolist()

In [113]:
categories = [x for x in labels if str(x) != 'nan']

In [114]:
train = train.reset_index(drop='True')
test = test.reset_index(drop='True')

In [115]:
trainData=[]
trainLabels = []
testLabels = []
for id in train.index:
    trainData.append(train.iloc[id,2])
    trainLabels.append(train.iloc[id,3])

testData=[]
for id in test.index:
    testData.append(test.iloc[id,2])
    testLabels.append(test.iloc[id,3])

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

vectorizer = TfidfVectorizer(min_df=5,
                                 max_df = 0.8,
                                 sublinear_tf=True,
                                 use_idf=True)
train_vectors = vectorizer.fit_transform(trainData)
test_vectors = vectorizer.transform(testData)


classifier_rbf = svm.SVC()
classifier_rbf.fit(train_vectors, trainLabels)
prediction_rbf = classifier_rbf.predict(test_vectors)


In [117]:
classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(train_vectors, trainLabels)
prediction_linear = classifier_linear.predict(test_vectors)


In [118]:
classifier_liblinear = svm.LinearSVC()
classifier_liblinear.fit(train_vectors, trainLabels)
prediction_liblinear = classifier_liblinear.predict(test_vectors)

In [122]:
from sklearn.metrics import f1_score
f1_score(testLabels, prediction_linear, average='micro')  

0.57287822878228778